<hr style="border:0.02in solid gray"> </hr>

<center><font color=#555555><font size=6> Laboratorio 4: Microestructura del mercado </font> <br> <br>
    
<center><font color=#555555><font size=5> Microestructura de trading </font> <br> <br> <br>

<center><font color=#555555><font size=5> Cuauhtémoc Corrales Camacho, if720482@iteso.mx </font> <br> <br>

<center><font color=#555555><font size=5> Jesús Sebastián González Ortiz, if721621@iteso.mx </font> <br> <br> <br>

<center> <font color= #555555> <font size = 4> Noviembre 2022 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/SebasGlez2000/myst_lab4'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

<font color= #555555> <font size = 4> Abstract </font>

Con la información recaudada de tres diferentes exchanges de diferentes partes del mundo, se usaron los libros de ordenes de tres diferentes criptomonedas que son BTC, ETH y SOL, con la información de volumenes y precios, el objetivo será visualizar y modelar la microestructura del mercado por medio de sus comportamientos de los libros de ordenes y sus características.

<br>

# <font color= #6B6B6B> <font size = 6> 1. Introduction </font>

<hr style="border:0.02in solid gray"> </hr>

En el siguiente laboratorio se estará trabajando en la Microestructura del mercado enfocado en las criptomonedas del momento, se recaurdará información del Libro de Ordenes de criptomonedas de diversos exchanges internacionales, esto con el propósito de visualizar y modelar la microestructura del mercado por medio de su comportamiento del libro de ordenes. 

Los libros de ordenes de dichas criptomonedas serán descargadas de tres exchanges de diferentes partes del mundo para ver la variación de precios que hay entre dichas plataformas y así poder observar si es posible el arbitraje entre ellos. Los exchanges utilizados serán Bitso que es una plataforma mexicana, Gate que es una plataforma de las Islas Caimán y Okcoin que es una plataforma de Estados Unidos. 

Como se puede observar, nuestros exchanges tienen sedes muy alejadas unas de otras entonces las ordenes recibidas entre dichos exchanges van a variar bastante unas de otras, la descarga del libro de ordenes se hará de una hora y de ahí se partirá para obtener los datos que se estan buscando para la realización del laboratorio.

De los Libros de Ordenes (LO) de los exchanges se extraerán los siguientes datos:

- Ask-volume para cada timestamp del LO: El ask-volume es la cantidad de ordenes que existen en el mercado para la **venta** del activo en el timestamp indicado.
- Bid-volume para cada timestamp del LO: El bid-volume es la cantidad de ordenes que existen en el mercado para la **compra** del activo en el timestamp indicado.
- Total Volume (ask-bid) para cada timestamp del LO: Es la suma de los volumenes que hay entre el bid y el ask en el timestamp indicado.
- Mid-Price (promedio de Top Of the Book) para cada timestamp del LO: Es el promedio de los precios de la oferta y la demanda del activo al momento.
- VWAP para cada timestamp del LO: Es el precio medio ponderado por volumen, muestra la relación entre el precio de un activo y su volumen total de operaciones.

A partir de los precios recolectados entre el bid y el ask, se calculará el spread, que es la diferencia de precios entre la demanda (bid) y la oferta (ask) de un mercado, y se contruirán series de tiempo con la información recopilada y procesada para su fácil visualización.

Se dividirá en tres secciones el laboratorio; el primero consiste en la extracción del Libro de Ordenes y OHLC de la criptomonedas de diversos exchanges, después la contrsucción de las Series de Tiempo indicadas previamente y por último es el modelado de la microestructura que consiste en estimar el Effective Spread del modelo de Roll.

<br>

# <font color= #6B6B6B> 2. Descripcion de datos </font>

<hr style="border:0.02in solid gray"> </hr>

 La información obtenida para el libro de ordenes son de tres criptomonedas que se consideran que son de las más influyentes del mercado: Ethereum (ETH), Solana (SOL) y Bitcoin (BTC), dichas monedas se medirán en dólares americanos para una visión más generalizada del mercado. Cada par sera extraido de los tres diferentes exchanges propuestos durante la introduccion, con lo cual se dispondran de nueve instrumentos para su analisis. 

 Esta informacion sera descargada de la paqueteria ccxt, la cual pide en su documentacion un instrumento, un exchange y un limite de ordenes. Y arroja como salidas un diccionario con el nombre del instrumento, los niveles de operaciones de bid y ask dentro del limite establecido, un timestamp en formato de microsegundos y una fecha en formata iso. 

 Para el desarrollo de este laboratorio se necesita extraer una hora de datos, donde dicho periodo de tiempo recopilara todos los instrumentos dentro del intervalo.

In [8]:
import main as mn
import visualizations as vs
import data as dt

<br>

# <font color= #6B6B6B> 4. Procesamiento y Visualización de datos  </font>

<hr style="border:0.02in solid gray"> </hr>

El primer paso fue hacer una funcion de iteracion para almacenar los datos correspondientes a los nueve instrumentos en el periodo determinado de una hora. Dicha funcion tiene como entradas el limite de operaciones, el exchange y el tiempo asignado y arroja una lista con nueve grupos con los respectivos los datos en pares de segundos durante el intervalo. Dicha operacion de extraccion se realizo mediante la funcion cccxt.fetch_order_book.

Tras esto se creo la funcion de extracion de precios de cierre, la cual necesita de un exchange, un instrumento y un limite y arroja un diccionario que incluye los precios de cierre por minito en el limite escrito. La funcion utilizada para este metedo fue ccxt.fetch_ohlcv. Un dato curioso de este metodo es que extrae valores del datos mas nuevo hacia atras en el tiempo, en la longitud del limite ingresado. 

Con estas dos extracciones, se creo una funcion para crear un formato de salida como el especificado en la primera de las visualizaciones. 

## <font color= #6B6B6B> 4.1. Creacion de tabla de microestructura  </font>

La función llamada microestructura lee los datos descargados de las funciones anteriores para organizarlos en un DataFrame con el formato exigido en las indicaciones. Dicha funcion recibe como entrada una lista con los exchange, una lista con los instrumentos, el tiempo de intervalo y una lista de apoyo ya declarada al inicio del codigo. 

Dentro de esta funcion, se calcula el numero de niveles, el volumen total de operaciones *Bid* y *Ask*, asi como el volumen total operado; el precio medio del instrumento,  el cual se calcula mediante el promedio de los precios de *Bid* y *Ask* en su primer nivel; el VWAP se calcula mediante un promedio ponderado que implica los precios de *Bid* y *Ask* y sus respectivos volumenes. Finalmente se calcula el spread de precios para cada intervalo  y una columna de apoyo referente al tiempo del intervalo. 

In [9]:
tabla = mn.me
tabla.drop(['milliseconds','timestamp2','Spread'],axis=1)

,exchange,symbol,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,VWAP
0,Bitso,ETH/USD,2022-11-12T00:34:41.000Z,50,110.045558,61.639810,171.685367,1272.350,1275.197786
1,Bitso,ETH/USD,2022-11-12T00:34:47.000Z,50,104.213558,61.630610,165.844167,1271.600,1275.011486
2,Bitso,ETH/USD,2022-11-12T00:34:53.000Z,50,247.376680,61.630910,309.007589,1271.650,1274.922716
3,Bitso,ETH/USD,2022-11-12T00:34:59.000Z,50,234.005527,68.164506,302.170032,1272.250,1272.778383
4,Bitso,ETH/USD,2022-11-12T00:35:05.000Z,50,234.348980,68.200026,302.549006,1271.900,1272.132110
...,...,...,...,...,...,...,...,...,...
184,Okcoin,BTC/USD,2022-11-12T00:36:22.356Z,50,48.920800,59.316500,108.237300,16920.305,16918.490000
185,Okcoin,BTC/USD,2022-11-12T00:36:28.648Z,50,48.926400,59.488100,108.414500,16920.305,16917.481667
186,Okcoin,BTC/USD,2022-11-12T00:36:34.719Z,50,50.088600,64.774300,114.862900,16920.305,16916.147000
187,Okcoin,BTC/USD,2022-11-12T00:36:40.380Z,50,50.446000,64.650700,115.096700,16927.060,16923.376667


## <font color= #6B6B6B> 4.2. Gráficas de instrumentos  </font>

Con la tabla de microestructura creada, es posible graficar el comportamiento de cada activo para el intervalo elegido. Para este caso, se graficara los cambios en el **Mid Price** o precio medio para cada uno de los nueve instrumentos. 

In [10]:
vs.grafos(mn.me,dt.exchl)

El siguiente paso fue construir un formato para el calculo de spread efectivo, para el cual se definio una nueva funcion denominada como spread_cut, el cual recibia como entradas la tabla de microestructura y la tabla con los precios de cierre y arroja como salidas una lista con nueve tablas, las cuales tienen datos de timestamp en minutos, el precio de cierre y el spread real. 

La clave para este inciso fue establecer una union adecuada entre ambas tablas con el proposito de que cada activo y precio de cierre en la tabla de precios de cierre recibiera el spread real dentro de la tabla de micrestructura, ademas que se realizo una transformacion para que el timestamp de las salidas quedara en dentro del valor de minutos y no de segundos. 

Tras esto, se crearon dos funciones para el calculo del spread efectivo, la primera hace el calculo bruto del spread nuevo mediante la tabla obtenida en la funcion de microestructura, mientras que la segunda añade los resultados de la primera funcion y los concatena a las salidas de la funcion spread_cut mencionada anteriormente. 

<br>

# <font color= #6B6B6B> <font size = 6> 5. Resultados </font>

<hr style="border:0.02in solid gray"> </hr>

Como salidas del nuestros procesos, se obtienen nueve tablas, las cuales almacenan la informacion del timestamp en su indice, el precio de cierre en su primera columna, el spread real en su segunda columna y el resultado del calculado del spread efectivo en su tercera columna. 

## <font color= #6B6B6B> 5.1 Resultados ETH/USD </font>

In [14]:
mn.Resultados[0].head()

,CP ETH/USD Bitso,Spread,Effective,CP ETH/USD Gate.io,Spread,Effective,CP ETH/USD OKCoin,Spread,Effective
timestamp2,,,,,,,,,
2022-11-12T00:34,1276.0,5.9,0.000000,1272.31,0.12,0.000000,1276.07,1.83,0.000000
2022-11-12T00:35,1276.0,6.6,1.171820,1274.87,0.82,0.432210,1274.10,1.35,0.321242
2022-11-12T00:36,1276.0,6.9,0.071297,1275.24,0.27,0.318444,1274.10,1.35,0.230184


## <font color= #6B6B6B> 5.2 Resultados SOL/USD </font>

In [15]:
mn.Resultados[1].head()

,CP SOL/USD Bitso,Spread,Effective,CP SOL/USD Gate.io,Spread,Effective,CP SOL/USD OKCoin,Spread,Effective
timestamp2,,,,,,,,,
2022-11-12T00:34,16.088,0.021,0.000000,16.093,0.041,0.000000,15.878,0.164,0.000000
2022-11-12T00:35,16.132,0.013,0.011719,16.103,0.002,0.010568,15.878,0.149,0.026906
2022-11-12T00:36,16.138,0.035,0.006576,16.093,0.041,0.006388,15.878,0.130,0.048058


## <font color= #6B6B6B> 5.3 Resultados BTC/USD </font>

In [13]:
mn.Resultados[2].head()

,CP BTC/USD Bitso,Spread,Effective,CP BTC/USD Gate.io,Spread,Effective,CP BTC/USD OKCoin,Spread,Effective
timestamp2,,,,,,,,,
2022-11-12T00:34,16890.0,11.0,0.000000,16900.23,0.51,0.000000,16898.99,11.28,0.000000
2022-11-12T00:35,16911.0,8.0,8.678386,16923.08,42.10,12.596021,16918.23,13.50,2.279081
2022-11-12T00:36,16923.0,18.0,10.422941,16929.21,0.53,19.406156,16925.52,7.61,0.947667


<br>

# <font color= #6B6B6B> <font size = 6> 6. References </font>

<hr style="border:0.02in solid gray"> </hr>

- VWAP (definición). (s. f.). IG. https://www.ig.com/es/glosario-trading/definicion-de-vwap

- elEconomista.es. (s. f.). Spread: qué es - Diccionario de Economía. https://www.eleconomista.es/diccionario-de-economia/spread

- Nine, E. N. (2022, 7 enero). El precio BID, el MID y el ASK. ¿Qué son y para qué se usan? ». Ninety Nine. https://ninetynine.com/blog/bid-mid-ask/

<br>